In [2]:
import pandas as pd
import plotly.express as px


In [51]:
# Importeer het dataframe met de punten die gegeven zijn per songfestival
euro_df = pd.read_csv('eurovision_song_contest_1975_2019.csv')
euro_df.columns=[column.strip() for column in euro_df.columns]
euro_df.head()

,Year,(semi-) final,Edition,Jury or Televoting,From country,To country,Points,Duplicate
0,1975,f,1975f,J,Belgium,Belgium,0,x
1,1975,f,1975f,J,Belgium,Finland,0,NaN
2,1975,f,1975f,J,Belgium,France,2,NaN
3,1975,f,1975f,J,Belgium,Germany,0,NaN
4,1975,f,1975f,J,Belgium,Ireland,12,NaN


In [94]:
def total_points(dataframe):
    """
    Itereert door het eurovisie dataframe en maakt een dictionary met daarin alle landen
    als key en de punten die de landen hebben gehaald in de vorm van een lijst als value.
    Vervolgens berekent de functie het gemiddelde van de totale punten. 
    Er wordt een pandas dataframe met de landen als kolom en de gemiddelde punten teruggegeven.
    """
    # Maak een lege dictionary aan
    points = {}

    # Itereer over de dataframe
    for index, row in dataframe.iterrows():

        # Controleer of het om een finale gaat
        # Zo niet, ga dan naar het volgende item in de dataframe
        if row[1] == 'f':

            # Als het land al in de dictionary staat, moeten de punten worden
            # toegevoegd aan de lijst
            if row[5] in points:
                points[row[5]].append(row[6])

            # Als het land nog niet in de dictionary staat moet deze worden
            # toegevoegd met de punten
            else:
                points.update({row[5]:[row[6]]})
    
    # Maak een nieuwe dictionary om de gemiddelde punten per land in te kunnen doen
    totals = {}

    # Itereer over de points dictionary
    # Bereken de gemiddeldes van de lijst met punten
    for country in points:
        totals.update({country:sum(points[country])/len(points[country])})

    # Return een dataframe op basis van de totals dictionary
    return pd.DataFrame([{"Country":country, "Points":points} for country, points in totals.items()])

average_points = total_points(euro_df)


In [114]:
# Maak een figuur van de wereldkaart met de gemiddelde punten die landen gehaald hebben in
# de finales van het songfestival
average_points_fig = px.choropleth(average_points,
                    locationmode='country names', 
                    locations='Country', 
                    color='Points',
                    title='Gemiddelde aantal punten dat landen hebben gekregen in de finale',
                    height=1000)

# Laat het figuur zien
average_points_fig.show()

In [111]:
def total_per_year(dataframe):
    total_year = {}
    
    for index, row in dataframe.iterrows():
        if row[1] == 'f':
            if row[5] in total_year:
                if row[0] in total_year[row[5]]:
                    total_year[row[5]][row[0]] += row[6]
                else:
                    total_year[row[5]].update({row[0]: row[6]})
            else:
                total_year.update({row[5]: {row[0]:row[6]}})
    
    total_per_year_df = pd.DataFrame({'Country':[], 'Year':[], 'Points':[]})

    for country, scores in total_year.items():
        for year, score in scores.items():
            total_per_year_df.loc[len(total_per_year_df)] = [country, year, score]

    return total_per_year_df

points_per_year = total_per_year(euro_df)


In [112]:
points_per_year['Year'] = pd.Categorical(points_per_year['Year'], ordered=True)
year_order = [year for year in range(1975, 2019)]

fig_per_year = px.choropleth(points_per_year, 
                    locationmode='country names', 
                    locations='Country', 
                    color='Points',
                    animation_frame='Year',
                    category_orders={'Year':year_order},
                    title='Totale aantal punten dat een land in de finale heeft gekregen per jaar',
                    height=1000)
fig_per_year.show()